In [1]:

import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing     import MinMaxScaler
import seaborn as sns

from sklearn.model_selection   import KFold
from sklearn.svm               import SVR


#Classification Libraries
from sklearn.neighbors         import KNeighborsClassifier
from sklearn                   import metrics
from sklearn.model_selection   import train_test_split

from sklearn.tree              import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.tree              import export_graphviz
from sklearn.externals.six     import StringIO  
from IPython.display           import Image  
import pydotplus

from sklearn.linear_model      import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier


#file_pointer = open("Output.txt", "w+")

def get_dummy_feature(df, column_name):
    
    df[column_name] = df[column_name].astype('object')

    dummies=pd.get_dummies(df[column_name], prefix=column_name) 
    
    df=pd.concat([df,dummies],axis=1) 
    
    df.drop(column_name,axis=1,inplace=True)
    
    return df 


def data_normalize(df):
    
    
    catcols = [ 'ACCOM_TYPE', 'CUSTOMER_CATEGORY','RISK_CATEGORY','SEX', 'MARITAL_STATUS', 'OCCUPN_CODE']
    
    for col in catcols:
        df = get_dummy_feature(df , col )
    
    #numcols = ['CUSTOMER_CODE', 'TIN_NUM_AVAILABLE', 'EXEMP_IN_TDS','ANNUAL_LIMIT',  'MONTHLY_LIMIT',  'AGE','BC_ANNUAL_INCOME', 'HAVE_INSUR_POLICY', 'PLAN_FOR_INSUR_POLICY' ]
    #catcols = ['CUSTOMER_TYPE','CUSTOMER_CATEGORY','CUSTOMER_SEGMENT_CODE','RISK_CATEGORY','SEX', 'MARITAL_STATUS', 'OCCUPN_CODE'  ]
    
    features = df.columns
    
    df = df.dropna()
    df = pd.DataFrame(df,columns=features)
    
    df.loc[(df.PRODUCTCATEGORY == 'Saving'),'PRODUCTCATEGORY']='1'
    df.loc[(df.PRODUCTCATEGORY == 'Current'),'PRODUCTCATEGORY']='2'
    df.loc[(df.PRODUCTCATEGORY == 'Fixed'),'PRODUCTCATEGORY']='3'
    df.loc[(df.PRODUCTCATEGORY == 'DPS'),'PRODUCTCATEGORY']='4'
    df.loc[(df.PRODUCTCATEGORY == 'Conti'),'PRODUCTCATEGORY']='5'
    df.loc[(df.PRODUCTCATEGORY == 'Term'),'PRODUCTCATEGORY']='6'
    
    df['PRODUCTCATEGORY'] = pd.to_numeric(df['PRODUCTCATEGORY'])
    
    df = pd.DataFrame(df,columns=features)
    X = df.copy()
    X = X.dropna()
    
    #print(X)
    return X


def fill_null_values(df, p_column_name):
    column_name = p_column_name
    df[column_name] = df[column_name].fillna(df[column_name].mode()[0])


def dataclean_null_values(df):
    for col in df.columns: 
        #fill_null_values(df, col)
        df[col] = df[col].fillna(df[col].mode()[0])
        
    return df 

def fatch_data_from_csv():
    file_path = "/content/drive/My Drive/MIT_1912/4th Sem/PROD_DATA.csv"
    df = pd.read_csv(file_path)
    return df


def create_featured_dataset(df):
    #print(df)
    del df['CUSTOMER_CODE']
    new_df = df[[#'BC_ANNUAL_INCOME', 
                 'AMOUNT_OF_CREDIT_TRAN', 'ACCOM_TYPE_1', 'AGE', 'AMOUNT_OF_DEBIT_TRAN', 'ANNUAL_INCOME_SLAB', 'NUMBER_OF_CREDIT_TRAN', 'NUMBER_OF_DEBIT_TRAN', 'OCCUPN_CODE_15', 'OCCUPN_CODE_5', 'TIN_NUM_AVAILABLE', 'MARITAL_STATUS_M', 'MARITAL_STATUS_S', 'OCCUPN_CODE_1', 'OCCUPN_CODE_12', 'OCCUPN_CODE_14', 'OCCUPN_CODE_24', 'OCCUPN_CODE_72', 'OCCUPN_CODE_77', 'OCCUPN_CODE_9', 'SEX_F', 'ACCOM_TYPE_3', 'EXEMP_IN_TDS', 'HAVE_INSUR_POLICY', 'OCCUPN_CODE_17', 'OCCUPN_CODE_22', 'OCCUPN_CODE_74', 'OCCUPN_CODE_99', 'SEX_M', 'BC_ANNUAL_INCOME', 'CUSTOMER_CATEGORY_2', 'PLAN_FOR_INSUR_POLICY', 'RISK_CATEGORY_1', 'RISK_CATEGORY_3', 'ACCOM_TYPE_6', 'RISK_CATEGORY_2', 'PRODUCTCATEGORY']].copy() 

    return new_df 


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def LogisticRegressionClassification(df):
    print("I am LogisticRegressionClassification")
    from imblearn.combine import SMOTETomek
    from imblearn.over_sampling import SMOTE
    from sklearn.metrics import accuracy_score
    
    X=df.drop('PRODUCTCATEGORY',axis=1)
    y=df.PRODUCTCATEGORY
    
    #smk = SMOTETomek(random_state=42)
    #X,y=smk.fit_sample(X,y)


    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)

    cv = KFold(n_splits=10, random_state=42, shuffle=True)
    #feature_cols = ['EXEMP_IN_TDS', 'MARITAL_STATUS_S', 'AGE', 'OCCUPN_CODE', 'SEX_F', 'SEX_T', 'CUSTOMER_CATEGORY', 'CUSTOMER_SEGMENT_CODE', 'MARITAL_STATUS_M', 'SEX_M', 'HAVE_INSUR_POLICY', 'TIN_NUM_AVAILABLE', 'PLAN_FOR_INSUR_POLICY', 'RISK_CATEGORY']
    
    for train_index, test_index in cv.split(X):

        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]

        #sm = SMOTE(random_state = 33)
        #X_train_new, y_train_new = sm.fit_sample(X_train, y_train.ravel())
        

        logreg = LogisticRegression(solver = 'lbfgs', class_weight = 'balanced')
        logreg.fit(X_train,y_train)

        # prediction for Training data
        train_pred_sm = logreg.predict(X_train)

        # prediction for Testing data
        test_pred_sm = logreg.predict(X_test)


        #y_pred = clf.predict(X_test) 
        print('Accuracy score for Training Dataset = ', accuracy_score(train_pred_sm, y_train))
        print('Accuracy score for Testing Dataset = ', accuracy_score(test_pred_sm, y_test))    
        #print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

        print('Accuracy report for Training')
        print(metrics.classification_report(train_pred_sm, y_train, digits=3))

        print('Accuracy report for Testing')
        print( metrics.classification_report(y_test, test_pred_sm, digits=3))

        
        '''
        cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
        #% matplotlib inline
        class_names=['1','2','3','4','5','6'] # name  of classes
        
        fig, ax = plt.subplots()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names)
        plt.yticks(tick_marks, class_names)
        
        # create heatmap
        sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
        ax.xaxis.set_label_position("top")
        plt.tight_layout()
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
        plt.Text(0.5,257.44,'Predicted label')
        
        print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
        print(metrics.classification_report(y_test, y_pred, digits=3))
        '''
        '''
        print("Precision:",metrics.precision_score(y_test, y_pred))
        print("Recall:",metrics.recall_score(y_test, y_pred))
        
        y_pred_proba = logreg.predict_proba(X_test)[::,1]
        fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
        auc = metrics.roc_auc_score(y_test, y_pred_proba)
        plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
        plt.legend(loc=4)
        plt.show()
        '''
        

In [0]:
def run():
    
    df = fatch_data_from_csv()
    df = dataclean_null_values(df)
    #print(df)
    df = data_normalize(df)
    #print(df)
    new_df = create_featured_dataset(df)
    #print(new_df)
    
    ############### Classification test #####################
    
    #KNN_Classification(new_df)
    
    #DecisionTreeClassification(new_df)
    LogisticRegressionClassification(new_df)
    #RandomForestClassifier(new_df)
    #Naive_Bayes_Classification(new_df)
    #AdaBoost_Classification(new_df)

In [16]:
if __name__ == '__main__':
    run()

I am LogisticRegressionClassification


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of itera

Accuracy score for Training Dataset =  0.3203009926352866
Accuracy score for Testing Dataset =  0.32164641467748906
Accuracy report for Training
              precision    recall  f1-score   support

           1      0.304     0.879     0.451     53233
           2      0.845     0.051     0.096     21354
           3      0.192     0.045     0.074     17826
           4      0.402     0.146     0.214     36474
           5      0.277     0.030     0.055     17707
           6      0.433     0.135     0.206     40786

    accuracy                          0.320    187380
   macro avg      0.409     0.214     0.182    187380
weighted avg      0.399     0.320     0.238    187380

Accuracy report for Testing
              precision    recall  f1-score   support

           1      0.876     0.308     0.455     17110
           2      0.053     0.874     0.099       143
           3      0.041     0.156     0.065       508
           4      0.144     0.402     0.212      1427
           5 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of itera

Accuracy score for Training Dataset =  0.31987234564870504
Accuracy score for Testing Dataset =  0.31777137367915465
Accuracy report for Training
              precision    recall  f1-score   support

           1      0.303     0.878     0.451     53120
           2      0.845     0.051     0.097     21439
           3      0.185     0.046     0.074     17025
           4      0.406     0.144     0.212     37261
           5      0.260     0.029     0.052     17340
           6      0.436     0.134     0.205     41196

    accuracy                          0.320    187381
   macro avg      0.406     0.214     0.182    187381
weighted avg      0.400     0.320     0.238    187381

Accuracy report for Testing
              precision    recall  f1-score   support

           1      0.880     0.301     0.449     17085
           2      0.046     0.899     0.088       119
           3      0.046     0.186     0.074       468
           4      0.143     0.410     0.212      1475
           5

...

Accuracy score for Training Dataset =  0.32040068096551944
Accuracy score for Testing Dataset =  0.31743515850144094
Accuracy report for Training
              precision    recall  f1-score   support

           1      0.304     0.881     0.452     53159
           2      0.843     0.050     0.094     21399
           3      0.180     0.044     0.071     17520
           4      0.404     0.145     0.214     36742
           5      0.278     0.030     0.055     17870
           6      0.434     0.136     0.207     40691

    accuracy                          0.320    187381
   macro avg      0.407     0.214     0.182    187381
weighted avg      0.399     0.320     0.238    187381

Accuracy report for Testing
              precision    recall  f1-score   support

           1      0.879     0.298     0.446     17119
           2      0.059     0.854     0.110       164
           3      0.049     0.208     0.079       466
           4      0.147     0.414     0.217      1459
           5

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of itera

Accuracy score for Training Dataset =  0.31775366766107555
Accuracy score for Testing Dataset =  0.3162343900096061
Accuracy report for Training
              precision    recall  f1-score   support

           1      0.301     0.880     0.448     52648
           2      0.846     0.051     0.096     21475
           3      0.188     0.045     0.072     18064
           4      0.402     0.143     0.211     36866
           5      0.272     0.030     0.055     17427
           6      0.435     0.135     0.206     40901

    accuracy                          0.318    187381
   macro avg      0.407     0.214     0.181    187381
weighted avg      0.399     0.318     0.236    187381

Accuracy report for Testing
              precision    recall  f1-score   support

           1      0.883     0.299     0.447     17121
           2      0.050     0.869     0.095       137
           3      0.043     0.204     0.071       432
           4      0.146     0.388     0.212      1504
           5 

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of itera

Accuracy score for Training Dataset =  0.32260474647909876
Accuracy score for Testing Dataset =  0.316954851104707
Accuracy report for Training
              precision    recall  f1-score   support

           1      0.307     0.879     0.455     53772
           2      0.848     0.051     0.097     21176
           3      0.185     0.045     0.072     17574
           4      0.400     0.144     0.212     36376
           5      0.289     0.030     0.054     18787
           6      0.430     0.138     0.209     39696

    accuracy                          0.323    187381
   macro avg      0.410     0.215     0.183    187381
weighted avg      0.399     0.323     0.239    187381

Accuracy report for Testing
              precision    recall  f1-score   support

           1      0.874     0.302     0.449     17081
           2      0.048     0.807     0.091       145
           3      0.040     0.167     0.065       467
           4      0.146     0.389     0.212      1503
           5  